In [31]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats

# Loading Data

In [20]:
df = (
    pd.read_csv('./data/icu_ibd_all_table.csv')
    .assign(intime = lambda x: pd.to_datetime(x['intime']))
)

print(df.shape)
print(df.head())

(1161, 32)
    hadm_id  subject_id              intime              outtime       los  \
0  20004577    14583219 2164-07-15 04:11:42  2164-07-15 21:30:20  0.721273   
1  20007037    16147352 2129-11-14 14:43:18  2129-11-15 16:22:06  1.068610   
2  20030062    14887253 2127-06-05 14:10:01  2127-06-07 14:50:49  2.028330   
3  20033189    14685940 2127-07-31 14:46:00  2127-08-01 20:58:34  1.258730   
4  20033924    13158454 2196-04-28 11:28:47  2196-05-01 14:22:54  3.120910   

   mortality  gender  age  weight   bmi  ...  hemoglobin  CRP  \
0          0       0   30     NaN   NaN  ...         NaN  NaN   
1          0       1   58  153.00   NaN  ...        11.0  NaN   
2          1       0   57     NaN   NaN  ...        10.3  NaN   
3          1       0   76  147.30  28.8  ...        10.5  NaN   
4          1       0   64  184.08  33.7  ...         7.4  NaN   

                     race  language  marital_status  insurance  die_in_icu  \
0                   WHITE   ENGLISH         MARRIED

In [21]:
df2 = (
    pd.read_csv('./data/patients_ibd.csv')
    .assign(anchor_year=lambda x: pd.to_datetime(x['anchor_year'].astype(str) + '-01-01'))
)

print(df2.shape)
print(df2.head())

(2417, 6)
   subject_id gender  anchor_age anchor_year anchor_year_group  dod
0    10098672      M          61  2140-01-01       2011 - 2013  NaN
1    10303503      F          23  2144-01-01       2008 - 2010  NaN
2    10312715      M          39  2176-01-01       2008 - 2010  NaN
3    10318500      F          46  2194-01-01       2011 - 2013  NaN
4    10410021      M          49  2135-01-01       2011 - 2013  NaN


# Preprocess Data

## Make a Combination

In [22]:
data = (
    df.merge(
        df2[['subject_id', 'anchor_year']], 
        on='subject_id', 
        how='left',
    ) # 需要 df2 的 anchor_year 信息
    .sort_values(by=['subject_id', 'intime'],) # 每个 subject_id 按照入院时间 intime 排序
    .assign(
        age = lambda x: ((x['intime'] - x['anchor_year']).dt.days) / 365 + x['age']
    )
    .drop(columns=['outtime', 'anchor_year'])
)

print(data.shape)
print(data.head())

(1161, 31)
       hadm_id  subject_id              intime      los  mortality  gender  \
327   22643604    10024331 2141-03-18 19:36:08  4.10571          1       1   
1022  28899194    10025647 2176-09-22 17:57:15  1.96810          1       1   
896   27617929    10037975 2185-01-17 19:12:12  4.87824          1       1   
110   20845468    10048262 2168-08-21 00:21:53  0.44588          0       1   
908   27715453    10056223 2122-09-23 15:08:45  5.04106          0       1   

            age  weight  bmi  heart_rate  ...  hemoglobin  CRP  \
327   73.210959     NaN  NaN        70.0  ...         9.4  NaN   
1022  84.726027     NaN  NaN        70.0  ...        10.5  NaN   
896   60.043836     NaN  NaN        90.0  ...        12.5  NaN   
110   46.641096     NaN  NaN       101.0  ...         NaN  NaN   
908   50.728767     NaN  NaN        89.0  ...         9.4  NaN   

                             race  language  marital_status  insurance  \
327                         WHITE   ENGLISH      

## Delete Missing Value and Useless Columns

In [23]:
# 检查缺失值
cols_missing = []

for col in data.columns:
    if data[col].isna().sum() / data.shape[0]*100 > 5:
        cols_missing.append(col)
        print(
            f'{col} => '
            f'NA Count: {data[col].isna().sum()} ({data[col].isna().sum() / data.shape[0]*100:.2f}%)'
            '\n'
        )

print(cols_missing)

weight => NA Count: 1066 (91.82%)

bmi => NA Count: 1070 (92.16%)

systolic_pressure => NA Count: 1099 (94.66%)

diastolic_pressure => NA Count: 1099 (94.66%)

temperature => NA Count: 962 (82.86%)

white_blood_cell => NA Count: 1155 (99.48%)

red_blood_cell => NA Count: 705 (60.72%)

CRP => NA Count: 1161 (100.00%)

['weight', 'bmi', 'systolic_pressure', 'diastolic_pressure', 'temperature', 'white_blood_cell', 'red_blood_cell', 'CRP']


In [24]:
# 删除不需要的列
cols_useless = [
    'hadm_id',
    'mortality',
    # 'die_in_icu',
]

cols_except = cols_missing + cols_useless

data.drop(columns=cols_except, inplace=True)

print(data.shape)
print(data.head())

(1161, 21)
      subject_id              intime      los  gender        age  heart_rate  \
327     10024331 2141-03-18 19:36:08  4.10571       1  73.210959        70.0   
1022    10025647 2176-09-22 17:57:15  1.96810       1  84.726027        70.0   
896     10037975 2185-01-17 19:12:12  4.87824       1  60.043836        90.0   
110     10048262 2168-08-21 00:21:53  0.44588       1  46.641096       101.0   
908     10056223 2122-09-23 15:08:45  5.04106       1  50.728767        89.0   

      respiratory_rate  hematocrit   rdw  platelet  ...   mch  hemoglobin  \
327               16.0        29.1  12.8     151.0  ...  30.2         9.4   
1022              19.0        31.2  15.3     192.0  ...  27.7        10.5   
896               37.0        38.1  16.9      16.0  ...  30.7        12.5   
110               26.0         NaN   NaN       NaN  ...   NaN         NaN   
908               20.0        26.1  15.4      77.0  ...  31.0         9.4   

                             race language ma

## Drop Rows with Too Much Missing Value 

In [25]:
data = data[~(data.isna().sum(axis=1) > (data.shape[1]+1)*0.05)]

print(data.subject_id.unique().shape)
print(data.shape)
print(data.head())

(652,)
(1127, 21)
      subject_id              intime      los  gender        age  heart_rate  \
327     10024331 2141-03-18 19:36:08  4.10571       1  73.210959        70.0   
1022    10025647 2176-09-22 17:57:15  1.96810       1  84.726027        70.0   
896     10037975 2185-01-17 19:12:12  4.87824       1  60.043836        90.0   
908     10056223 2122-09-23 15:08:45  5.04106       1  50.728767        89.0   
559     10063856 2174-03-08 01:04:16  1.37475       0  56.183562        43.0   

      respiratory_rate  hematocrit   rdw  platelet  ...   mch  hemoglobin  \
327               16.0        29.1  12.8     151.0  ...  30.2         9.4   
1022              19.0        31.2  15.3     192.0  ...  27.7        10.5   
896               37.0        38.1  16.9      16.0  ...  30.7        12.5   
908               20.0        26.1  15.4      77.0  ...  31.0         9.4   
559               17.0        29.6  12.7     274.0  ...  31.9        10.3   

                             race lang

## Formatting Race

In [26]:
def parse_race(race):
    if 'WHITE' in race:
        return 'WHITE'
    elif 'BLACK' in race:
        return 'BLACK'
    elif 'HISPANIC' in race or 'LATINO' in race:
        return 'HISPANIC/LATINO'
    elif 'ASIAN' in race:
        return 'ASIAN'
    else:
        return 'OTHER'


data.loc[:, 'race'] = data.loc[:, 'race'].apply(parse_race)

In [27]:
data

,subject_id,intime,los,gender,age,heart_rate,respiratory_rate,hematocrit,rdw,platelet,...,mch,hemoglobin,race,language,marital_status,insurance,die_in_icu,uc_only,cd_only,uc_cd
327,10024331,2141-03-18 19:36:08,4.105710,1,73.210959,70.0,16.0,29.1,12.8,151.0,...,30.2,9.4,WHITE,ENGLISH,MARRIED,Medicare,0,1,0,0
1022,10025647,2176-09-22 17:57:15,1.968100,1,84.726027,70.0,19.0,31.2,15.3,192.0,...,27.7,10.5,WHITE,ENGLISH,MARRIED,Medicare,0,1,0,0
896,10037975,2185-01-17 19:12:12,4.878240,1,60.043836,90.0,37.0,38.1,16.9,16.0,...,30.7,12.5,OTHER,ENGLISH,MARRIED,Medicare,1,0,1,0
908,10056223,2122-09-23 15:08:45,5.041060,1,50.728767,89.0,20.0,26.1,15.4,77.0,...,31.0,9.4,HISPANIC/LATINO,?,SINGLE,Medicaid,0,1,0,0
559,10063856,2174-03-08 01:04:16,1.374750,0,56.183562,43.0,17.0,29.6,12.7,274.0,...,31.9,10.3,WHITE,ENGLISH,MARRIED,Other,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,19970491,2129-05-17 17:57:50,46.912200,1,55.372603,78.0,29.0,36.8,14.0,243.0,...,34.7,12.4,WHITE,ENGLISH,MARRIED,Medicaid,0,1,0,0
259,19970491,2131-02-11 01:38:42,3.689210,1,57.112329,94.0,26.0,35.7,13.3,279.0,...,32.0,11.9,WHITE,ENGLISH,MARRIED,Medicaid,0,1,0,0
116,19970491,2131-05-10 19:23:44,2.840370,1,57.353425,78.0,14.0,22.8,14.9,446.0,...,29.0,7.5,WHITE,ENGLISH,MARRIED,Medicaid,0,1,0,0
176,19970491,2131-07-18 08:02:07,0.204433,1,57.542466,72.0,12.0,28.5,13.7,229.0,...,31.2,9.4,WHITE,ENGLISH,MARRIED,Medicaid,0,1,0,0


## Formatting Dummy Variables

In [28]:
data.drop(columns=['uc_cd'], inplace=True)

In [29]:
# 处理 dummy variables
cols = ['language', 'race', 'marital_status', 'insurance']

for col in cols:
    tmp = data[col].value_counts()
    res = pd.DataFrame({'index': tmp.index, 'count': tmp.values, 'percentage': tmp.values/tmp.sum()*100})
    print(f'{res.set_index("index")}')
    print()

         count  percentage
index                     
ENGLISH   1073   95.208518
?           54    4.791482

                 count  percentage
index                             
WHITE              927   82.253771
BLACK              102    9.050577
OTHER               70    6.211180
HISPANIC/LATINO     21    1.863354
ASIAN                7    0.621118

          count  percentage
index                      
MARRIED     512   46.001797
SINGLE      369   33.153639
WIDOWED     137   12.309075
DIVORCED     95    8.535490

          count  percentage
index                      
Medicare    524   46.495120
Other       516   45.785271
Medicaid     87    7.719610



In [30]:
data = pd.get_dummies(data=data, 
                      columns=cols, 
                      prefix=cols, 
                      drop_first=True)

data.columns

Index(['subject_id', 'intime', 'los', 'gender', 'age', 'heart_rate',
       'respiratory_rate', 'hematocrit', 'rdw', 'platelet', 'mcv', 'mch',
       'hemoglobin', 'die_in_icu', 'uc_only', 'cd_only', 'language_ENGLISH',
       'race_BLACK', 'race_HISPANIC/LATINO', 'race_OTHER', 'race_WHITE',
       'marital_status_MARRIED', 'marital_status_SINGLE',
       'marital_status_WIDOWED', 'insurance_Medicare', 'insurance_Other'],
      dtype='object')

## Standalization

In [32]:
cols = ['age', 'heart_rate', 'respiratory_rate', 'hematocrit', 'rdw', 'platelet', 'mcv', 'mch', 'hemoglobin',]

# scaler = StandardScaler()
scaler = MinMaxScaler()

data[cols] = scaler.fit_transform(data[cols])

In [33]:
data

,subject_id,intime,los,gender,age,heart_rate,respiratory_rate,hematocrit,rdw,platelet,...,language_ENGLISH,race_BLACK,race_HISPANIC/LATINO,race_OTHER,race_WHITE,marital_status_MARRIED,marital_status_SINGLE,marital_status_WIDOWED,insurance_Medicare,insurance_Other
327,10024331,2141-03-18 19:36:08,4.105710,1,0.684717,0.28750,0.32,0.425234,0.484848,0.127773,...,True,False,False,False,True,True,False,False,True,False
1022,10025647,2176-09-22 17:57:15,1.968100,1,0.833860,0.28750,0.38,0.474299,0.579545,0.164153,...,True,False,False,False,True,True,False,False,True,False
896,10037975,2185-01-17 19:12:12,4.878240,1,0.514176,0.41250,0.74,0.635514,0.640152,0.007986,...,True,False,False,True,False,True,False,False,True,False
908,10056223,2122-09-23 15:08:45,5.041060,1,0.393528,0.40625,0.40,0.355140,0.583333,0.062112,...,False,False,True,False,False,False,True,False,False,False
559,10063856,2174-03-08 01:04:16,1.374750,0,0.464178,0.11875,0.34,0.436916,0.481061,0.236912,...,True,False,False,False,True,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,19970491,2129-05-17 17:57:50,46.912200,1,0.453674,0.33750,0.58,0.605140,0.530303,0.209406,...,True,False,False,False,True,True,False,False,False,False
259,19970491,2131-02-11 01:38:42,3.689210,1,0.476207,0.43750,0.52,0.579439,0.503788,0.241349,...,True,False,False,False,True,True,False,False,False,False
116,19970491,2131-05-10 19:23:44,2.840370,1,0.479330,0.33750,0.28,0.278037,0.564394,0.389530,...,True,False,False,False,True,True,False,False,False,False
176,19970491,2131-07-18 08:02:07,0.204433,1,0.481779,0.30000,0.24,0.411215,0.518939,0.196983,...,True,False,False,False,True,True,False,False,False,False


In [34]:
# 转换所有 bool 类型为 int
for k, v in data.dtypes.items():
    if v == bool:
        data[k] = data[k].astype(int)

# Export Data

## For Statistics

In [38]:
# 统计学分析
(
    data.merge(
        df.loc[:, ['subject_id', 'race', 'language', 'marital_status', 'insurance']],
        on='subject_id', how='left'
    )
    .assign(race = lambda x: parse_race(x['race']))
    # .drop_duplicates(subset='subject_id', keep='first')
    .to_csv('./data_processed/data_stats_.csv')
)

## First ICU Record

In [42]:
(
    data
    # .drop_duplicates(subset='subject_id', keep='first',)
    # .drop(columns=['die_in_icu'])
    .to_csv('./data_processed/data_first_record_.csv')
)

## Dead in ICU

In [37]:
(
    data[data.die_in_icu == 1]
    .drop(columns=['die_in_icu'])
    .to_csv('./data_processed/data_die_in_icu.csv')
)